In [59]:
import os
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import time

In [60]:
def openDataForProg(progdataDir, trialsToUse=[0,1,2,3,4], problemSize='medprob'):
    pava = 'PA'
    # let's open all the CSV files in the given directory
    csvs = []
    for trial in trialsToUse:
        csvs = csvs + list(glob.glob(progdataDir+'/'+'*-'+problemSize+'*-cntrVariab-trial'+str(trial)+'*-'+pava+'-traces/*.csv'))

    #print('Reading CSVs:\n', '\n'.join([os.path.basename(x) for x in csvs]))

    df = pd.DataFrame()
    for idx, csv in enumerate(csvs):
        print(csv, 'reading csv', idx+1, 'of', len(csvs), end='')
        # get the counter name from the directory
        startidx = csv.find('PAPI_')
        endidx = csv.find('-cntrVariab')
        counter = csv[startidx:endidx]
        #print('\n using counter', counter)

        starttime = time.time()
        rawdf = pd.read_csv(csv, sep=' ')
        rawdf['counter'] = counter
        df = df.append(rawdf)
        print(' -- Read in', time.time()-starttime, 'seconds')
        
    return df[['region', 'globalidx', 'policy', 'f0', 'counter']]

In [61]:
def statifyDF(df):
    # let's group the dataframe elements and get the avrg and stddevs of each group
    newdf = df.groupby(['region', 'globalidx', 'policy', 'counter'])['f0'].mean().reset_index().sort_values(by=['globalidx', 'policy', 'counter'])
    newdf['avrg'] = newdf['f0']


    newdf['std'] = df.groupby(['region', 'globalidx', 'policy', 'counter'])['f0'].std().reset_index().sort_values(by=['globalidx', 'policy', 'counter'])['f0']
    newdf['std2'] = df.groupby(['region', 'globalidx', 'policy', 'counter'])['f0'].std().reset_index().sort_values(by=['globalidx', 'policy', 'counter'])['f0']*2
    
    newdf = newdf[['region', 'globalidx', 'policy', 'counter', 'avrg', 'std', 'std2']]

    newdf['coeffVar'] = newdf['std']/newdf['avrg']
    newdf['coeffVar'] = newdf['coeffVar'].fillna(0)
    
    return newdf

In [ ]:
%%time
ftdf = None
spdf = None
cfddf = None

if not os.path.exists('ftdf.csv'):
    ftdf  = openDataForProg('/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data')
    ftdf  = statifyDF(ftdf)
    ftdf.to_csv('ftdf.csv', index=False)
else:
    ftdf = pd.read_csv('ftdf.csv')

if not os.path.exists('spdf.csv'):
    spdf  = openDataForProg('/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/sp-data')
    spdf  = statifyDF(spdf)
    spdf.to_csv('spdf.csv', index=False)
else:
    spdf = pd.read_csv('spdf.csv')
    
if not os.path.exists('cfddf.csv'):
    cfddf = openDataForProg('/g/g15/bolet1/workspace/benchmarks/rodinia/rodinia_3.1/openmp/cfd/cfd-data')
    cfddf = statifyDF(cfddf)
    cfddf.to_csv('cfddf.csv', index=False)
else:
    cfddf = pd.read_csv('cfddf.csv')

/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_DP_OPS-cntrVariab-trial0-PA-traces/trace-Static,policy=0-region-ft.c.apollo.region.l199-rank-0.csv reading csv 1 of 1200 -- Read in 0.013417243957519531 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_DP_OPS-cntrVariab-trial0-PA-traces/trace-Static,policy=0-region-ft.c.apollo.region.l376-rank-0.csv reading csv 2 of 1200 -- Read in 0.00917959213256836 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_DP_OPS-cntrVariab-trial0-PA-traces/trace-Static,policy=0-region-ft.c.apollo.region.l263-rank-0.csv reading csv 3 of 1200 -- Read in 0.008669614791870117 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_DP_OPS-cntrVariab-trial0-PA-traces/trace-Static,policy=0-region-ft.c.apollo.region.l447-rank-0.csv reading csv 4 of 1200 -- Read in 0.009715318679

/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_SP_OPS-cntrVariab-trial0-PA-traces/trace-Static,policy=2-region-ft.c.apollo.region.l199-rank-0.csv reading csv 41 of 1200 -- Read in 0.011537551879882812 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_SP_OPS-cntrVariab-trial0-PA-traces/trace-Static,policy=2-region-ft.c.apollo.region.l376-rank-0.csv reading csv 42 of 1200 -- Read in 0.010383129119873047 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_SP_OPS-cntrVariab-trial0-PA-traces/trace-Static,policy=2-region-ft.c.apollo.region.l263-rank-0.csv reading csv 43 of 1200 -- Read in 0.009452581405639648 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_SP_OPS-cntrVariab-trial0-PA-traces/trace-Static,policy=2-region-ft.c.apollo.region.l447-rank-0.csv reading csv 44 of 1200 -- Read in 0.0109529

/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_SR_INS-cntrVariab-trial0-PA-traces/trace-Static,policy=0-region-ft.c.apollo.region.l676-rank-0.csv reading csv 80 of 1200 -- Read in 0.010370969772338867 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_SR_INS-cntrVariab-trial0-PA-traces/trace-Static,policy=1-region-ft.c.apollo.region.l199-rank-0.csv reading csv 81 of 1200 -- Read in 0.010940313339233398 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_SR_INS-cntrVariab-trial0-PA-traces/trace-Static,policy=1-region-ft.c.apollo.region.l376-rank-0.csv reading csv 82 of 1200 -- Read in 0.009899616241455078 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_SR_INS-cntrVariab-trial0-PA-traces/trace-Static,policy=1-region-ft.c.apollo.region.l263-rank-0.csv reading csv 83 of 1200 -- Read in 0.0107569

/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_LST_INS-cntrVariab-trial0-PA-traces/trace-Static,policy=2-region-ft.c.apollo.region.l223-rank-0.csv reading csv 119 of 1200 -- Read in 0.010371208190917969 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_LST_INS-cntrVariab-trial0-PA-traces/trace-Static,policy=2-region-ft.c.apollo.region.l676-rank-0.csv reading csv 120 of 1200 -- Read in 0.008945226669311523 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_BR_INS-cntrVariab-trial0-PA-traces/trace-Static,policy=0-region-ft.c.apollo.region.l199-rank-0.csv reading csv 121 of 1200 -- Read in 0.010200262069702148 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_BR_INS-cntrVariab-trial0-PA-traces/trace-Static,policy=0-region-ft.c.apollo.region.l376-rank-0.csv reading csv 122 of 1200 -- Read in 0.0

/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_MEM_WCY-cntrVariab-trial0-PA-traces/trace-Static,policy=2-region-ft.c.apollo.region.l199-rank-0.csv reading csv 161 of 1200 -- Read in 0.010012626647949219 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_MEM_WCY-cntrVariab-trial0-PA-traces/trace-Static,policy=2-region-ft.c.apollo.region.l376-rank-0.csv reading csv 162 of 1200 -- Read in 0.009825944900512695 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_MEM_WCY-cntrVariab-trial0-PA-traces/trace-Static,policy=2-region-ft.c.apollo.region.l263-rank-0.csv reading csv 163 of 1200 -- Read in 0.009659767150878906 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_MEM_WCY-cntrVariab-trial0-PA-traces/trace-Static,policy=2-region-ft.c.apollo.region.l447-rank-0.csv reading csv 164 of 1200 -- Read in 0

...

/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_TOT_INS-cntrVariab-trial3-PA-traces/trace-Static,policy=0-region-ft.c.apollo.region.l513-rank-0.csv reading csv 942 of 1200 -- Read in 0.022011518478393555 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_TOT_INS-cntrVariab-trial3-PA-traces/trace-Static,policy=0-region-ft.c.apollo.region.l223-rank-0.csv reading csv 943 of 1200 -- Read in 0.010642290115356445 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_TOT_INS-cntrVariab-trial3-PA-traces/trace-Static,policy=0-region-ft.c.apollo.region.l676-rank-0.csv reading csv 944 of 1200 -- Read in 0.010894775390625 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_TOT_INS-cntrVariab-trial3-PA-traces/trace-Static,policy=1-region-ft.c.apollo.region.l199-rank-0.csv reading csv 945 of 1200 -- Read in 0.02

/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_SP_OPS-cntrVariab-trial4-PA-traces/trace-Static,policy=0-region-ft.c.apollo.region.l199-rank-0.csv reading csv 985 of 1200 -- Read in 0.011642932891845703 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_SP_OPS-cntrVariab-trial4-PA-traces/trace-Static,policy=0-region-ft.c.apollo.region.l376-rank-0.csv reading csv 986 of 1200 -- Read in 0.010422706604003906 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_SP_OPS-cntrVariab-trial4-PA-traces/trace-Static,policy=0-region-ft.c.apollo.region.l263-rank-0.csv reading csv 987 of 1200 -- Read in 0.010814189910888672 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_SP_OPS-cntrVariab-trial4-PA-traces/trace-Static,policy=0-region-ft.c.apollo.region.l447-rank-0.csv reading csv 988 of 1200 -- Read in 0.009

/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_LD_INS-cntrVariab-trial4-PA-traces/trace-Static,policy=1-region-ft.c.apollo.region.l223-rank-0.csv reading csv 1023 of 1200 -- Read in 0.012315034866333008 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_LD_INS-cntrVariab-trial4-PA-traces/trace-Static,policy=1-region-ft.c.apollo.region.l676-rank-0.csv reading csv 1024 of 1200 -- Read in 0.010628700256347656 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_LD_INS-cntrVariab-trial4-PA-traces/trace-Static,policy=2-region-ft.c.apollo.region.l199-rank-0.csv reading csv 1025 of 1200 -- Read in 0.010174036026000977 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_LD_INS-cntrVariab-trial4-PA-traces/trace-Static,policy=2-region-ft.c.apollo.region.l376-rank-0.csv reading csv 1026 of 1200 -- Read in 0

/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_LST_INS-cntrVariab-trial4-PA-traces/trace-Static,policy=0-region-ft.c.apollo.region.l447-rank-0.csv reading csv 1060 of 1200 -- Read in 0.01133108139038086 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_LST_INS-cntrVariab-trial4-PA-traces/trace-Static,policy=0-region-ft.c.apollo.region.l480-rank-0.csv reading csv 1061 of 1200 -- Read in 0.011976480484008789 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_LST_INS-cntrVariab-trial4-PA-traces/trace-Static,policy=0-region-ft.c.apollo.region.l513-rank-0.csv reading csv 1062 of 1200 -- Read in 0.011544466018676758 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_LST_INS-cntrVariab-trial4-PA-traces/trace-Static,policy=0-region-ft.c.apollo.region.l223-rank-0.csv reading csv 1063 of 1200 -- Read i

/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_BR_INS-cntrVariab-trial4-PA-traces/trace-Static,policy=2-region-ft.c.apollo.region.l199-rank-0.csv reading csv 1097 of 1200 -- Read in 0.01221609115600586 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_BR_INS-cntrVariab-trial4-PA-traces/trace-Static,policy=2-region-ft.c.apollo.region.l376-rank-0.csv reading csv 1098 of 1200 -- Read in 0.011487483978271484 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_BR_INS-cntrVariab-trial4-PA-traces/trace-Static,policy=2-region-ft.c.apollo.region.l263-rank-0.csv reading csv 1099 of 1200 -- Read in 0.01085805892944336 seconds
/g/g15/bolet1/workspace/benchmarks/NPB/SNU_NPB_2019/NPB3.3-OMP-C/bin/ft-data/ft-medprob-PAPI_BR_INS-cntrVariab-trial4-PA-traces/trace-Static,policy=2-region-ft.c.apollo.region.l447-rank-0.csv reading csv 1100 of 1200 -- Read in 0.0

In [ ]:
print(ftdf.head())
print(spdf.head())
print(cfddf.head())

In [ ]:
# make a unified df
ftdf['prog'] = 'ft'
spdf['prog'] = 'sp'
cfddf['prog'] = 'cfd'

df = pd.concat([ftdf, spdf, cfddf], ignore_index=True)

In [ ]:
print(df.shape)
print(df.columns)

# what are the counters we can choose from?
counters = list(df['counter'].unique())
print('\n'.join(counters))

In [ ]:
# now let's plot the counter values alongside the CoV CDF plots
def plotCounter(df, counter, numbins=100):
    
    # get only the data for this counter
    alldata = df[df['counter'] == counter]
    
    # count how many progs we're working with
    progs = list(alldata['prog'].unique())
    numprogs = len(progs)
    
    # we're going to make a plot for each program
    fig, axs = plt.subplots(numprogs,2)
    fig.set_size_inches(18, 18)
    fig.set_dpi(300)
    fig.suptitle('['+counter+'] Counter Value and Variation Analysis Plots', fontsize=16)
    
    # for each program, let's make its plots
    for idx, prog in enumerate(progs):
        progdata = alldata[alldata['prog'] == prog]
        ax1 = axs[idx, 0]
        ax2 = axs[idx, 1]
        
        # the left side plot will be the counter values relative to global index
        p0data = progdata[(progdata['policy'] == 0)][['avrg', 'globalidx']]
        p1data = progdata[(progdata['policy'] == 1)][['avrg', 'globalidx']]
        p2data = progdata[(progdata['policy'] == 2)][['avrg', 'globalidx']]

        ax1.scatter(p0data['globalidx']-0.25, p0data['avrg'], color='red', label='policy0', s=2)
        ax1.scatter(p1data['globalidx']+0,    p1data['avrg'], color='green', label='policy1', s=2)
        ax1.scatter(p2data['globalidx']+0.25, p2data['avrg'], color='blue', label='policy2', s=2)

        ax1.set_title('['+prog+'] Counter Values During Execution with ['+counter+']')
        ax1.set_xlabel('Global Index')
        ax1.set_ylabel('Counter Value')
        #ax1.legend()
        
        # now let's plot the CDF CoV data
        p0data = progdata[(progdata['policy'] == 0)]['coeffVar']
        p1data = progdata[(progdata['policy'] == 1)]['coeffVar']
        p2data = progdata[(progdata['policy'] == 2)]['coeffVar']

        p0hist, p0edges = np.histogram(p0data, bins=numbins)
        p1hist, p1edges = np.histogram(p1data, bins=numbins)
        p2hist, p2edges = np.histogram(p2data, bins=numbins)

        p0cum = 100*p0hist.cumsum()/np.sum(p0hist)
        p1cum = 100*p1hist.cumsum()/np.sum(p1hist)
        p2cum = 100*p2hist.cumsum()/np.sum(p2hist)
        
        # let's calculate the 90% and 95% coverage points of each policy
        # let's find where the 90 and 95 marks happen
        #p090 = np.percentile(p0data, 90)
        #p095 = np.percentile(p0data, 95)
        #p190 = np.percentile(p1data, 90)
        #p195 = np.percentile(p1data, 95)
        #p290 = np.percentile(p2data, 90)
        #p295 = np.percentile(p2data, 95)

        ax2.plot(p0edges[:-1], p0cum, color='red', label='policy0')
        ax2.plot(p1edges[:-1], p1cum, color='green', label='policy1')
        ax2.plot(p2edges[:-1], p2cum, color='blue', label='policy2')
        
        #ax2.scatter([p090, p095, p190, p195, p290, p295], [90, 95, 90, 95, 90, 95], color='black', s=3)

        print(p0data.min(), p1data.min(), p2data.min())
        print(p0data.max(), p1data.max(), p2data.max())

        ax2.set_title('['+prog+'] CoV Cumulative Plot with ['+counter+']')
        ax2.set_xlabel('Coefficient of Variation')
        ax2.set_ylabel('Percent CDF')
        #ax2.legend()
    
    plt.legend()
    plt.show()
    return

In [ ]:
for counter in counters:
    plotCounter(df, counter)

In [ ]:
# now let's look at the distribution of stds across policies for this program

def plotCounterCDF(df, counter, numbins, progname):
    fig1, ax1 = plt.subplots()
    fig1.set_size_inches(18, 10)
    fig1.set_dpi(500)

    alldata = df[df['counter'] == counter]
    
    #bins = np.linspace(alldata['coeffVar'].min()*1.2, alldata['coeffVar'].max()*1.2, numbins)
    
    p0data = alldata[(alldata['policy'] == 0)]['coeffVar']
    p1data = alldata[(alldata['policy'] == 1)]['coeffVar']
    p2data = alldata[(alldata['policy'] == 2)]['coeffVar']
    
    p0hist, p0edges = np.histogram(p0data, bins=numbins)
    p1hist, p1edges = np.histogram(p1data, bins=numbins)
    p2hist, p2edges = np.histogram(p2data, bins=numbins)
    
    p0cum = 100*p0hist.cumsum()/np.sum(p0hist)
    p1cum = 100*p1hist.cumsum()/np.sum(p1hist)
    p2cum = 100*p2hist.cumsum()/np.sum(p2hist)
    
    ax1.plot(p0edges[:-1], p0cum, color='red', label='policy0')
    ax1.plot(p1edges[:-1], p1cum, color='green', label='policy1')
    ax1.plot(p2edges[:-1], p2cum, color='blue', label='policy2')
    
    print(p0data.min(), p1data.min(), p2data.min())
    print(p0data.max(), p1data.max(), p2data.max())

    ax1.set_title(progname+' STDev Cumulative Plot for: '+counter)
    ax1.set_xlabel('Coefficient of Variation')
    ax1.set_ylabel('Percent CDF')

    plt.legend()
    plt.show()
    return

In [ ]:
# now let's look at the distribution of stds across policies for this program

def plotCounterAverages(df, counter, progname):
    fig1, ax1 = plt.subplots()
    fig1.set_size_inches(18, 10)
    fig1.set_dpi(500)

    alldata = df[df['counter'] == counter]
    
    #bins = np.linspace(alldata['coeffVar'].min()*1.2, alldata['coeffVar'].max()*1.2, numbins)
    
    p0data = alldata[(alldata['policy'] == 0)][['avrg', 'globalidx']]
    p1data = alldata[(alldata['policy'] == 1)][['avrg', 'globalidx']]
    p2data = alldata[(alldata['policy'] == 2)][['avrg', 'globalidx']]
    
    ax1.scatter(p0data['globalidx']-0.25, p0data['avrg'], color='red', label='policy0')
    ax1.scatter(p1data['globalidx']+0,    p1data['avrg'], color='green', label='policy1')
    ax1.scatter(p2data['globalidx']+0.25, p2data['avrg'], color='blue', label='policy2')
    
    ax1.set_title(progname+' Counter Values During Execution: '+counter)
    ax1.set_xlabel('Global Index')
    ax1.set_ylabel('Counter Value')

    plt.legend()
    plt.show()
    return

In [ ]:
# now let's look at the actual counter values across policies
# we're hoping to find that at least one counter is pretty consistent
# across the policies.

print(newdf.columns)

In [ ]:
for counter in counters:   
    plotCounterAverages(newdf, counter, 'rodinia_cfd')

In [ ]:
for counter in counters:   
    plotCounterCDF(newdf, counter, 100, 'nas_sp')